# DETERMINE R function of $\theta$

- I dati nel .txt sono separati dal carattere '\t  ', disposti in 2 colonne relativi al tempo (in millisecondi) e alla funzione di correlazione

- $\sigma_{g^2}$ = 0.001 l'incertezza è fissata all'ultima ciffra significativa

- i dati validi da tenere in considerazione sono quelli nell'intervallo 0.1ms-10ms

- Gli angoli nel nome del file sono l'angolo supplementare di $\theta$ (per l'errore di misurazione)

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val

nel caricamento dei dati nelle strutture numpy i millisecondi vengono convertiti in secondi

In [2]:
# filename = f'./data2/g_{θ}_2'
# read = pd.read_table(filename, skiprows=30, nrows=245 - 30)
# # read.head()

# # print(read[read.columns[4]])
# print(read.loc[:, read.columns[2:11]].std(axis=1))

In [3]:
# Carico i dati {θ: {t: [], g: [], sigmag: []}}

data = {}

TETA = [40,50,60,70,80,90,100,110,120,130] 
GAMMA = []
sigmaGAMMA = []

# t convertito in secondi, si considerano dati solo tra 0.1 e 10 ms a causa del rumore
for θ in TETA:
    filename = f'./data2/g_{θ}_2.asc'
    
    read = pd.read_csv(filename, skiprows=30, nrows=245 - 30, sep='\t', encoding='ISO-8859-1')

    # print(read.columns[2:12])

    read = read[(read['Lag [ms]'] >= 0.1) & (read['Lag [ms]'] <= 10)]
    read['std'] = read.loc[:, read.columns[2:12]].std(axis=1) / np.sqrt(10 - 1)
    
    t = read['Lag [ms]'].to_numpy()/1000
    g = read['Average'].to_numpy()
    σg = read['std'].to_numpy()

    # θ = 180 - θ
    data[θ] = {'t':t,'g':g,'sigmag': σg} # converto in secondi

# TETA = 180 - TETA # inverto angoli supplementari
TETA = np.array(TETA)
# print(data)

In [4]:
# plt.errorbar(data[90]['t'],data[90]['g'],yerr=data[90]['sigmag'],fmt='',label='θ = 90°')
# plt.xlabel('Lag [s]')
# plt.xscale('log')
# plt.xlim(0.0001,0.010)
# plt.ylim(0,0.1)
# plt.ylabel('g(τ)')
# plt.legend()
# plt.show()

In [5]:
# Constants
n = 1.33 # index of refraction
sigman = 0.01 # uncertainty of the index of refraction
η = 10**-9  # g/(nm*s) viscosity of water
sigmaη = 0.01 * 10**-9  # uncertainty of the viscosity of water
λ = 632.8  # nm wavelength of the laser
T = 293  # K temperature
sigmaT = 1  # K uncertainty of the temperature
KB = 1.3806e-2  # g (nm)^2 / s^2 K boltzmann constant

# INTERPOLAZIONE

attraverso al funzione: $g^2(\tau) = A e^{-\gamma \tau} + B$

In [6]:
# fit data
CHI_r = []
DOF = []
PVALUE = []

f = lambda x, A, B, γ: A*np.exp(-γ*x) + B

for key in data.keys():
    x = data[key]['t']
    y = data[key]['g']
    sigmay = data[key]['sigmag']
    m = Interpolazione(x, y, sigmay, f, [0.05, 0.001, 2.7],['A','B','γ'])
    data[key]['fit'] = m
    # print(m.values['γ'])
    data[key]['gamma'] = m.values['γ']
    GAMMA.append(m.values['γ'])
    sigmaGAMMA.append(m.errors['γ'])
    CHI_r.append(m.rchi2)
    DOF = m.dof
    PVALUE.append(m.pvalue)

GAMMA = np.array(GAMMA) 
sigmaGAMMA = np.array(sigmaGAMMA)


In [7]:
AsA = {}
for θ in data.keys():
    A = data[θ]['fit'].values['A']
    sigmaA = data[θ]['fit'].errors['A']
    B = data[θ]['fit'].values['B']
    sigmaB = data[θ]['fit'].errors['B']
    AsA[θ] = {'A':A,'σA':sigmaA,'B':B,'σB':sigmaB}

Asa = pd.DataFrame(AsA)
display(Asa)

,40,50,60,70,80,90,100,110,120,130
A,0.078358,0.073376,0.076864,0.072878,0.077385,0.073206,0.073449,0.073629,0.068494,0.068616
σA,0.000166,0.000105,0.000148,0.000167,0.000227,0.000303,0.000331,0.000356,0.000410,0.000441
B,0.001716,0.000603,0.000226,0.000233,0.000106,0.000204,0.000074,0.000020,0.000078,0.000101
σB,0.000102,0.000047,0.000044,0.000042,0.000034,0.000034,0.000028,0.000029,0.000029,0.000028


In [8]:
# print FIT RESULT:
GAMMAFVAL = [final_val(g,sg,2,3,'Hz') for g,sg in zip(GAMMA,sigmaGAMMA)]
gammateta = pd.DataFrame({'θ':TETA,'γ':GAMMA,'σγ':sigmaGAMMA,'γ_out':GAMMAFVAL,'pvalue':PVALUE,'chir':CHI_r,'dof':DOF})

print('---------------------------FIT RESULT----------------------------')
display(gammateta)

# print(gammateta['γ'].to_numpy())
# print(gammateta['σγ'].to_numpy())

---------------------------FIT RESULT----------------------------


,θ,γ,σγ,γ_out,pvalue,chir,dof
0,40,527.056265,3.161627,(0.53 ± 0.0)e3 Hz,0.01,1.54,50
1,50,829.623716,2.342004,(0.83 ± 0.0)e3 Hz,0.00,3.11,50
2,60,1197.095239,3.706795,(1.2 ± 0.0)e3 Hz,0.01,1.50,50
3,70,1621.361270,6.085032,(1.62 ± 0.01)e3 Hz,0.01,1.56,50
4,80,2008.897986,8.552204,(2.01 ± 0.01)e3 Hz,0.02,1.46,50
5,90,2448.430520,12.517416,(2.45 ± 0.01)e3 Hz,0.86,0.79,50
6,100,2792.436701,15.497471,(2.79 ± 0.02)e3 Hz,0.01,1.48,50
7,110,3274.390615,19.240284,(3.27 ± 0.02)e3 Hz,0.02,1.43,50
8,120,3660.831510,22.275566,(3.66 ± 0.02)e3 Hz,0.01,1.51,50
9,130,4009.109054,28.030692,(4.01 ± 0.03)e3 Hz,0.17,1.19,50


# PLOT FIT

In [9]:
colors = px.colors.sequential.Turbo

fig = go.Figure()
for i,θ in enumerate(data.keys()):
    x = data[θ]['t']
    y = data[θ]['g']
    γ = data[θ]['gamma']
    sigmay = data[θ]['sigmag']
    X,Y = data[θ]['fit'].draw()

    # plt.plot(X,Y)
    # plt.xlim(0,1)
    # plt.show()

    fig.add_trace(go.Scatter(x=X, y=Y, mode='lines',line_color=colors[i], showlegend=False,hoverinfo='skip'))

    fig.add_trace(go.Scatter(x=x,y=y, mode='markers', name=f'θ={θ}°',
        marker=dict(color = colors[i]),
        error_y=dict( 
			type='data', 
			array=sigmay, 
			color=colors[i],
			thickness=1.5, 
			width=3, 
		)))

fig.update_layout(
        xaxis_type="log",
        yaxis_title="g<sup>2</sup>(τ)",
        xaxis_title="t (s)",
        # title='g<sup>2</sup>(τ) vs t',
        xaxis=dict(range=[np.log10(0.0001), np.log10(0.010)]),
        width=800,
        height=600,
        # title={
        #     'text': "Correlation Function g<sup>2</sup>(τ)",
        #     'y':0.9,
        #     'x':0.5,
        #     'xanchor': 'center',
        #     'yanchor': 'top',
        #     'font':dict(size=30)
        # },
        plot_bgcolor='white',
        font=dict(
            #family="Courier New, monospace",
            size=18,
            color="Black"
        ),
            # yaxis=dict(range=[0, 0.06])
    )

fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

fig.show()

In [10]:
# scarto = []
# for θ in data.keys():
#     y = data[θ]['g']
#     y_teo = f(data[θ]['t'],*data[θ]['fit'].values.values())
#     scarto.append((y - y_teo)**2)

# scarto = np.array(scarto)

# fig, axs = plt.subplots(2, 5, figsize=(20, 10))
# colors = plt.cm.viridis(np.linspace(0, 1, len(data.keys())))

# for ax, (θ, color) in zip(axs.flatten(), zip(data.keys(), colors)):
#     ax.hist(scarto[TETA.tolist().index(θ)], bins=30, color=color, edgecolor='black')
#     ax.set_title(f'θ = {θ}°')

# plt.tight_layout()
# plt.show()

$Q = \frac{4\pi n}{\lambda}\sin\frac{\theta}{2}$

$\gamma = 2DQ^2 \Rightarrow D = \frac{\gamma\lambda^2}{32\pi^2n^2\sin^2\frac{\theta}{2}}$

$D = \frac{K_B T}{6\pi\eta\R}$

$R = \frac{16\pi n^2 K_B T}{3\eta\lambda^2\gamma}\sin^2\frac{\theta}{2}$

n = 1.33

$\eta$ = 0.01poise

$\lambda$ = 633nm

$K_B$ = 1.3806 $\cdot 10^{-2} g(nm)^2s^{-2}K^{-1}$

T = 293K

In [11]:
# fit gamma teta

def gamma_2DQ2(θ,n2D):
    theta = np.deg2rad(θ)
    return 2*n2D*16*(np.pi**2)*np.sin(theta/2)**2/(λ**2)

gammatetafit = Interpolazione(TETA, GAMMA, sigmaGAMMA, gamma_2DQ2, [100000*n**2],['n2D'])
print(gammatetafit)

n2D,σn2D = gammatetafit.values['n2D'],gammatetafit.errors['n2D']

D = n2D/(n**2)
σD = D*np.sqrt((σn2D/n2D)**2 + (sigman/n)**2)

print('n2D = coefficiente diffusione * n^2 = ',final_val(n2D,σn2D,2,exp=6,udm='nm^2/s'))
print('D = coefficiente diffusione = ',final_val(D,σD,2,exp=6,udm='m^2/s'))

----------------- VALORI FIT: -----------------
n2D: (6.059 ± 0.008)e6 

dof: 9
chi2: 344.29
chi2 ridotto: 38.25
pvalue: 0.0
------------------------------------------------

n2D = coefficiente diffusione * n^2 =  (6.06 ± 0.01)e6 nm^2/s
D = coefficiente diffusione =  (3.43 ± 0.03)e6 m^2/s


In [12]:
# plot fit gamma teta

fig = go.Figure()

X,Y = gammatetafit.draw()
fig.add_trace(go.Scatter(x=X, y=Y, mode='lines',line_color=px.colors.sequential.Plasma[6], showlegend=False,hoverinfo='skip'))

fig.add_trace(go.Scatter(x=TETA,y=GAMMA, mode='markers',showlegend=False,
    marker=dict(color=px.colors.sequential.Plasma[0]),
    error_y=dict( 
        type='data', 
        array=sigmaGAMMA, 
        color=px.colors.sequential.Plasma[1],
        thickness=1.5, 
        width=3, 
    )))

fig.update_layout(
    # xaxis_type="log",
    yaxis_title="γ (Hz)",
    xaxis_title="θ (°)",
    # title='g<sup>2</sup>(τ) vs t',
    # xaxis=dict(range=[np.log10(0.0001), np.log10(0.010)]),
    width=600,
    height=400,
    # title={
    #     'text': "γ = 2DQ<sup>2</sup>(θ)",
    #     'y':0.8,
    #     'x':0.5,
    #     'xanchor': 'center',
    #     'yanchor': 'top',
    #     'font':dict(size=15)
    # },
    plot_bgcolor='white',
    font=dict(
        #family="Courier New, monospace",
        size=10,
        color="Black"
    ),
        # yaxis=dict(range=[0, 0.06])
)

fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
# fig.update_layout(font=dict(size=10))
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

fig.show()

In [13]:
# def calculate_R(gamma, theta):
#     theta_rad = np.radians(theta)
#     R = (16 * np.pi * n**2 * T * KB * np.sin(theta_rad / 2)**2) / (3 * η * λ**2 * gamma)
#     return R

# def calculate_sigma_R(gamma, sigma_gamma,sigmaT,sigmaη,sigman, theta):
#     theta_rad = np.radians(theta)
#     term1 = (16 * np.pi * n**2 * T * KB * np.sin(theta_rad / 2)**2) / (3 * η * λ**2 * gamma**2) * sigma_gamma
#     term2 = (16 * np.pi * n**2 * KB * np.sin(theta_rad / 2)**2) / (3 * η * λ**2 * gamma) * sigmaT
#     term3 = (16 * np.pi * n**2 * T * KB * np.sin(theta_rad / 2)**2) / (3 * η**2 * λ**2 * gamma) * sigmaη
#     term4 = (32 * np.pi * n * T * KB * np.sin(theta_rad / 2)**2) / (3 * η * λ**2 * gamma) * sigman
#     sigma_R = np.sqrt(term1**2 + term2**2 + term3**2 + term4**2)
#     return sigma_R

# # Calculate sigma_R for each gamma and theta
# sigma_R_values = np.array([calculate_sigma_R(gamma, sigma_gamma,sigmaT,sigmaη,sigman, theta) for gamma, sigma_gamma, theta in zip(GAMMA, sigmaGAMMA, TETA)])

# # Calculate R for each gamma and theta
# R_values = np.array([calculate_R(gamma, theta) for gamma, theta in zip(GAMMA, TETA)])
# R_out = np.array([final_val(r,sr,1,udm='nm') for r,sr in zip(R_values,sigma_R_values)])
# R_values_df = pd.DataFrame({'θ (deg)': TETA, 'R (nm)': R_values,'σR (nm)': sigma_R_values,'R_out (nm)':R_out})
# display(R_values_df)

In [14]:
# R = np.average(R_values,weights=1/sigma_R_values**2)

# σR = (1/np.sqrt(np.sum(1/sigma_R_values**2)))


# # from statsmodels.stats.weightstats import DescrStatsW
# # #calculate weighted standard deviation
# # σR = DescrStatsW(R_values, weights=1/sigma_R_values**2, ddof=1).std#
# # # σR = np.std(R_values,ddof=1,mean=R)

# print('R = ',final_val(R,σR,1,udm='nm'))

# R CON COEFF DI DIFFUSIONE:

In [15]:
R = KB*T/(6*np.pi*η*D)
σR = R*np.sqrt((sigmaT/T)**2 + (sigmaη/η)**2 + (σD/D)**2)

print('R = ',final_val(R,σR,1,udm='nm'))

R =  62.6 ± 0.8 nm


# TEST COMPATIBILITA'

In [16]:
from scipy.stats import norm

costruttoreD = 120
sigmaDcostruttore = 15

D = R*2
σD = 2*σR

σ = np.sqrt(σD**2 + sigmaDcostruttore**2)

t = (D-costruttoreD)/σ


print('Diametro misurato: ',final_val(D,σD,1,0,udm='nm'))
print('Diametro costruttore: ',final_val(costruttoreD,sigmaDcostruttore,0,0,udm='nm'))
print(f"t = {np.round(t,3)}")
print(f"COmpatibilità: {np.round(norm.cdf(t)*100,1)}%")

# NOTA: il t non deve essere maggiore di 2, la probabilità deve essere minore possibile

Diametro misurato:  125.3 ± 1.6 nm
Diametro costruttore:  120 ± 15 nm
t = 0.351
COmpatibilità: 63.7%
